In [1]:
import sys
import pandas as pd
import fnmatch
import os

repo_dir = os.path.join(os.getcwd(), "../")
data_dir = os.path.join(repo_dir, 'data', 'sanborn_melanoma_2015')

# From Sanborn et. al. supplemental files
raw_data_fn = os.path.join(data_dir, "sanborn_melanoma_raw.xlsx")
PATIENTS = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Collect patient dataframes
raw_dfs = {}
for patient_id in PATIENTS:
    
    df = pd.read_excel(raw_data_fn, f"Patient {patient_id}")
    df['character_label'] = df.apply(lambda row: f"{row['Gene']}:{row['Chromosome']}:{row['Start position']}:{row['Alternate base']}", axis=1)
    # Drop rows where cn info is missing
    cn_cols = [x for x in df.columns if x.startswith("Copy number at base")]
    df = df.dropna(subset=cn_cols)
    raw_dfs[patient_id] = df
    print(patient_id, len(df))

/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


A 2109


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


B 135


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


...

/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


F 4696


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


G 767
H 3505


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Prepare inputs for pyclone clustering

In [2]:
pyclone_dir = os.path.join(data_dir, "pyclone_analysis")

def format_sample_name(name):
    return "_".join(name.lower().replace(",","").split())

def get_major_minor_cn(cn_str):
    items = cn_str.replace("(","").replace(")","").split(",")
    major_cn = int(float(items[0]))
    minor_cn = int(float(items[1]))
    return major_cn, minor_cn

patient_id_to_sample_names = {patient_id:[] for patient_id in PATIENTS}
pyclone_cols = ['mutation_id', 'ref_counts', 'var_counts', 'normal_cn', 'minor_cn', 'major_cn']
for patient_id,df in raw_dfs.items():
    print()
    print(patient_id)
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
    if not os.path.exists(os.path.join(pyclone_dir, patient_id)):
        os.makedirs(os.path.join(pyclone_dir, patient_id))
    data = []
    for sample in sample_names:
        cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
        subset = df[cols_to_keep]
        subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
        subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
        subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
        subset['var_counts'] = subset[cols_to_keep[2]] 
        subset['normal_cn'] = 2 # no sex chromosomes
        subset['mutation_id'] = subset[cols_to_keep[0]]
        subset = subset[pyclone_cols]
        print(subset)
        
        tsv_name = f"{patient_id}_{format_sample_name(sample)}.tsv"
        print(sample, tsv_name)
        subset.to_csv(os.path.join(pyclone_dir, patient_id, tsv_name), sep="\t")
        patient_id_to_sample_names[patient_id].append(format_sample_name(sample))
        
    print(sample_names)
    
    

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A val


A
              mutation_id  ref_counts  var_counts  normal_cn  minor_cn  \
0      RHBDF1:16:109459:A         211         123          2         1   
1     DEFB128:20:168662:T         231         117          2         1   
2     PLEKHG4B:5:171330:G         315         104          2         1   
3       BET1L:11:205338:A         188          95          2         1   
6       ATHL1:11:294638:T         481         284          2         1   
...                   ...         ...         ...        ...       ...   
2724   SMC4:3:160134105:C           7           8          2         1   
2725  KIF25:6:168442761:A          17          10          2         1   
2726   GRM6:5:178413186:A          10          26          2         0   
2728  CLCN2:3:184073266:A          68          49          2         1   
2729   MUC4:3:195513102:G           5           4          2         1   

      major_cn  
0            1  
1            1  
2            2  
3            1  
6            1  
...   

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['var_counts'] = subset[cols_to_keep[2]]
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

              mutation_id  ref_counts  var_counts  normal_cn  minor_cn  \
0      RHBDF1:16:109459:A         156         149          2         1   
1     DEFB128:20:168662:T         293         127          2         2   
2     PLEKHG4B:5:171330:G         341          92          2         2   
3       BET1L:11:205338:A         211          38          2         1   
6       ATHL1:11:294638:T         388         344          2         1   
...                   ...         ...         ...        ...       ...   
2724   SMC4:3:160134105:C          14           7          2         1   
2725  KIF25:6:168442761:A          13           9          2         1   
2726   GRM6:5:178413186:A          15          18          2         0   
2728  CLCN2:3:184073266:A          55          34          2         1   
2729   MUC4:3:195513102:G           0           1          2         1   

      major_cn  
0            2  
1            2  
2            4  
3            2  
6            2  
...      

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A val

...

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:28: SettingWithCopyWarning: 
A val

                  mutation_id  ref_counts  var_counts  normal_cn  minor_cn  \
0           SAMD11:1:874699:A          56          79          2         0   
1             AGRN:1:977029:T          42          35          2         0   
2           MORN1:1:2264193:T         101          78          2         0   
3            CHD5:1:6166505:A          30          35          2         0   
4          mir-34:1:9211833:A          82          66          2         0   
...                       ...         ...         ...        ...       ...   
4960        ITPR1:3:4718316:T          10           0          2         0   
4964      AP3S1:5:115205713:T          15           0          2         1   
4965  TMPRSS13:11:117789342:C          42           0          2         1   
4966   PITPNM2:12:123474377:T          11           0          2         1   
4967     RFXANK:19:19309965:T          40           0          2         1   

      major_cn  
0            2  
1            2  
2           

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['var_counts'] = subset[cols_to_keep[2]]
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

                mutation_id  ref_counts  var_counts  normal_cn  minor_cn  \
0         MXRA8:1:1290233:T         287          75          2         1   
1         TMC2:20:2542614:A         114          30          2         1   
2         TMC2:20:2593951:A         185          59          2         1   
3      FLYWCH1:16:2983242:T         200          47          2         2   
4        OR2C1:16:3406514:C         565         102          2         2   
...                     ...         ...         ...        ...       ...   
3612    PARP1:1:226564865:C          42           4          2         1   
3613    OBSCN:1:228430945:A          39           0          2         1   
3614     NGEF:2:233792627:A          91           0          2         1   
3615  ATG16L1:2:234173673:T          29           0          2         1   
3616   ARID4B:1:235341101:A          87           0          2         1   

      major_cn  
0            2  
1            3  
2            3  
3            2  
4 

/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['var_counts'] = subset[cols_to_keep[2]]
/scratch/lsftmp/3017671.tmpdir/ipykernel_21476/815694123.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

### Setup pyclone commands for each patient

In [3]:
# Load tumor purities
purities = pd.read_csv(os.path.join(data_dir, "sanborn_melanoma_purities.csv"))
purities['Site'] = purities.apply(lambda row: format_sample_name(row['Site']), axis=1)
purities

,Patient,Site,Tumor Purity,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,A,primary_forehead,76,NaN,NaN,NaN
1,A,parotid_metastasis,21,NaN,NaN,NaN
2,A,locoregional_skin_metastasis_1_forehead,42,NaN,NaN,NaN
3,A,locoregional_skin_metastasis_2_angle_jaw,62,NaN,NaN,NaN
4,B,primary_mid-left_back,89,NaN,NaN,NaN
5,B,lymph_node_metastasis_left_axilla,23,NaN,NaN,NaN
6,B,locoregional_skin_metastasis_1_left_back,88,NaN,NaN,NaN
7,B,locoregional_skin_metastasis_2_left_axilla,51,NaN,NaN,NaN
8,C,primary_right_lower_calf,54,NaN,NaN,NaN
9,C,locoregional_skin_metastasis_1_right_calf,85,NaN,NaN,NaN


In [4]:
for patient_id,df in raw_dfs.items():
    cmd = [f"bsub -n 8 -W 60:00 -R rusage[mem=8] -o output_{patient_id}.log -e error_{patient_id}.log", "PyClone run_analysis_pipeline", "--in_files"]
    patient_dir = os.path.join(pyclone_dir, patient_id)
    
    for sample_name in patient_id_to_sample_names[patient_id]:
        cmd.append(os.path.join(patient_dir,  f"{patient_id}_{sample_name}.tsv"))
    cmd += ["--working_dir", patient_dir, "--tumour_contents"]
    
    # Add tumour cell proportions for each sample
    for sample_name in patient_id_to_sample_names[patient_id]:
        purity = float(purities[(purities['Patient']==patient_id) &(purities['Site']==sample_name)]['Tumor Purity'].item())/100
        cmd.append(str(purity))
    # Add sample names
    cmd.append("--samples")
    for sample_name in patient_id_to_sample_names[patient_id]:
        cmd.append(sample_name)
    cmd += ["--max_clusters", "30", "--min_cluster_size", "5", "--num_iters", "10000", "--burnin", "5000"]
    print(" ".join(cmd))
    

bsub -n 8 -W 60:00 -R rusage[mem=8] -o output_A.log -e error_A.log PyClone run_analysis_pipeline --in_files /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/../data/sanborn_melanoma_2015/pyclone_analysis/A/A_primary_forehead.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/../data/sanborn_melanoma_2015/pyclone_analysis/A/A_parotid_metastasis.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/../data/sanborn_melanoma_2015/pyclone_analysis/A/A_locoregional_skin_metastasis_1_forehead.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/../data/sanborn_melanoma_2015/pyclone_analysis/A/A_locoregional_skin_metastasis_2_angle_jaw.tsv --working_dir /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/../data/sanborn_melanoma_2015/pyclone_analysis/A --tumour_contents 0.76 0.21 0.42 0.62 --samples primary_forehead parotid_metastasis locoregional_skin_metastasis_1_forehead locoregional_skin_metas

## Prepare data for orchard tree inference


In [9]:
from metient.util import data_extraction_util as dutil
import json
orchard_dir = os.path.join(data_dir, "orchard_trees")

for patient_id,df in raw_dfs.items():
    
    patient_pyclone_dir = os.path.join(pyclone_dir, patient_id)
    cluster_id_to_mut_names, mutation_names = dutil.load_pyclone_clusters(os.path.join(patient_pyclone_dir, "tables", "loci.tsv"), min_mut_thres=5)
    cluster_df = pd.read_csv(os.path.join(patient_pyclone_dir, "tables", "loci.tsv"), sep="\t")
    for cid in cluster_id_to_mut_names:
        print(cid, len(cluster_id_to_mut_names[cid]))
    header = ["id", "name", "var_reads", "total_reads", "var_read_prob"]
    mut_name_to_mut_id = {}
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
        
    with open(os.path.join(orchard_dir, f"{patient_id}.ssm"), "w") as f:
        f.write("\t".join(header))
        f.write("\n")  
        for i, mut in enumerate(mutation_names):
            mut_name_to_mut_id[mut] = f"m{i}"
            row = [f"m{i}", mut]
            
            var_reads = []
            total_reads = []
            var_read_probs = []
        
            for sample in sample_names:
                
                cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
                subset = df[cols_to_keep]
                subset = subset[subset['character_label']==mut]
                subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
                subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
            
                var = subset[cols_to_keep[2]].item()
                tot = subset[cols_to_keep[1]].item()
                major_cn = subset['major_cn'].item()
                minor_cn = subset['minor_cn'].item()
                formatted_sample_name = format_sample_name(sample)
                var_reads.append(str(var))
                total_reads.append(str(tot))
                p = float(purities[(purities['Patient']==patient_id) &(purities['Site']==formatted_sample_name)]['Tumor Purity'].item())/100
                x = (p*(major_cn+minor_cn)+2*(1-p))
                var_read_prob = (p*major_cn)/x
                var_read_probs.append(str(var_read_prob))

            row += [",".join(var_reads), ",".join(total_reads), ",".join(var_read_probs)]
            f.write("\t".join(row))
            f.write("\n")
    json_data = {"samples": sample_names, "clusters": [], "garbage": []}
    for x in range(0,len(cluster_id_to_mut_names)):
        json_data["clusters"].append([mut_name_to_mut_id[t] for t in cluster_id_to_mut_names[x]])

    with open(os.path.join(orchard_dir, f"{patient_id}.params.json"), 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False)

0 1973
1 47
2 8
3 44
4 6
5 15
6 5
0 78
1 6
2 9
3 8
4 8
0 1040
1 42
2 175
3 94
4 27
5 39
6 5
0 923
1 21
2 21
3 5
0 9
1 8
2 9
3 62
4 19
5 8
6 9
7 7
0 3666
1 434
2 200
3 84
4 11
5 49
6 14
7 150
8 7
9 14
10 18
11 14
12 9
0 627
1 23
2 13
3 15
4 30
5 19
6 10
0 141
1 2076
2 1151
3 58
4 8
5 11
6 28
7 25
8 5


In [10]:
# Orchard commands
for patient_id in raw_dfs:
    
    cmd = f"bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_{patient_id}.log -e error_{patient_id}.log"
    cmd += f" python /data/morrisq/divyak/projects/orchard/bin/orchard -p {patient_id}.ssm {patient_id}.params.json {patient_id}.results.npz"
    print(cmd)
    

bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_A.log -e error_A.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p A.ssm A.params.json A.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_B.log -e error_B.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p B.ssm B.params.json B.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_C.log -e error_C.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p C.ssm C.params.json C.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_D.log -e error_D.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p D.ssm D.params.json D.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_E.log -e error_E.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p E.ssm E.params.json E.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_F.log -e error_F.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p F.ssm F.params.json F.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8

## Take PyClone generated clusters and create csvs with ref and var counts pooled by cluster

In [11]:
# Need a tsv for each patient with ['anatomical_site_index','anatomical_site_label', 'character_index', 'character_label', 'ref', 'var', 'var_read_prob', 'site_category', 'num_mutations']
import re
import numpy as np


final_cols = ['anatomical_site_index','anatomical_site_label', 'character_index', 'character_label', 'ref', 'var', 'var_read_prob', 'site_category']
agg_rules = {}

for patient_id,df in raw_dfs.items():
    patient_pyclone_dir = os.path.join(pyclone_dir, patient_id)
    clstr_id_to_muts, mutation_names = dutil.load_pyclone_clusters(os.path.join(patient_pyclone_dir, "tables", f"loci.tsv"), min_mut_thres=5)
    mut_name_to_clstr_id = {}
    clstr_id_to_name = {}
    for cid in clstr_id_to_muts:
        for mut in clstr_id_to_muts[cid]:
            mut_name_to_clstr_id[mut] = cid
        clstr_id_to_name[cid] = ";".join(clstr_id_to_muts[cid])
        
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
    print("patient", patient_id, len(clstr_id_to_muts), "clusters", len(mutation_names), "mutations", len(sample_names), "samples")
    print(sample_names)
    
    data = []
    
    for midx, mut in enumerate(mutation_names):
        for sidx, sample in enumerate(sample_names):
            cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
            subset = df[cols_to_keep]
            subset = subset[subset['character_label']==mut]
            subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
            subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)

            var = subset[cols_to_keep[2]].item()
            tot = subset[cols_to_keep[1]].item()
            ref = tot - var
            major_cn = subset['major_cn'].item()
            minor_cn = subset['minor_cn'].item()
            formatted_sample_name = format_sample_name(sample)
            p = float(purities[(purities['Patient']==patient_id) &(purities['Site']==formatted_sample_name)]['Tumor Purity'].item())/100
            x = (p*(major_cn+minor_cn)+2*(1-p))
            var_read_prob = (p*major_cn)/x
            
            category = 'primary' if 'primary' in sample else 'metastasis'
            data.append([sidx, sample.capitalize(), midx, mut, ref, var, var_read_prob, category])
            
    patient_df = pd.DataFrame(data, columns=final_cols)

    dutil.write_pooled_tsv_from_clusters(patient_df, mut_name_to_clstr_id, clstr_id_to_name, agg_rules, 
                                         os.path.join(data_dir,"pyclone_clustered_tsvs"), f"patient{patient_id}")


patient A 7 clusters 2098 mutations 4 samples
['primary, forehead', 'parotid metastasis', 'locoregional skin metastasis 1, forehead', 'locoregional skin metastasis 2, angle jaw']
patient B 5 clusters 109 mutations 4 samples
['primary, mid-left back', 'lymph node metastasis, left axilla', 'locoregional skin metastasis 1, left back', 'locoregional skin metastasis 2, left axilla']
patient C 7 clusters 1422 mutations 3 samples
['primary, right lower calf', 'locoregional skin metastasis 1, right calf', 'locoregional skin metastasis 2, right mid-calf']
patient D 4 clusters 970 mutations 4 samples
['primary, right ankle', 'lymph node metastasis, right groin', 'locoregional skin metastasis 1, right ankle', 'locoregional skin metastasis 2, right leg']
patient E 8 clusters 131 mutations 5 samples
['primary, left heel', 'locoregional skin metastasis 1, left heel', 'locoregional skin metastasis 2, left heel', 'lymph node metastasis, left groin', 'locoregional skin metastasis 3, left heel']
patient

### Add the number of mutations column, update character_label column for display

In [12]:
for patient_id in raw_dfs:
    tsv_fn = os.path.join(data_dir,"pyclone_clustered_tsvs", f"patient{patient_id}_clustered_SNVs.tsv")
    df = pd.read_csv(tsv_fn, sep="\t")
    df['num_mutations'] = df.apply(lambda row: len(row['character_label'].split(";")), axis=1)
    df['full_label'] = df['character_label']
    df['character_label'] = df.apply(lambda row:dutil.get_pruned_mut_label(row['character_label'], ";", ":"), axis=1)
    df.to_csv(tsv_fn, sep="\t")